# Pickup to Delivery Overall

In [1]:
import os
import sys
import warnings
import pandas as pd
import numpy as np
import csv
import pickle
import matplotlib.pyplot as plt
from haversine import haversine, Unit
from sklearn.metrics.pairwise import haversine_distances, manhattan_distances
from sklearn.model_selection import train_test_split, cross_val_score

sys.path.insert(0, os.path.expanduser('./'))
import query_runner as qr
import utils

In [2]:
base_query_path = './queries/'
dwh_config, livedb_config, parameters_config = utils.load_config(config_file='./config.ini')
datalake_connection = qr.create_connection(db='datalake')
#monolith_connection = qr.create_connection(user=livedb_config['monolith_username'], password=livedb_config['monolith_password'], db='livedb')
#dispatching_db_connection = qr.create_connection(user=livedb_config['dispatching_db_username'], password=livedb_config['dispatching_db_password'], db='dispatchingdb')

In [3]:
start_date = parameters_config['start_date']
end_date = parameters_config['end_date']
country_code = parameters_config['country_code']
cities = parameters_config['cities']

print(f'Start date: {start_date} | End date: {end_date} | Countries: {country_code} | Cities: {cities}')

Start date: 2024-09-30 | End date: 2024-10-20 | Countries: ES | Cities: 'MAD', 'BCN', 'SEV', 'ALC'


In [4]:
parameters = {
    'start_date': start_date,
    'end_date': end_date,
    'country_code': country_code,
    'cities': cities
}

## Load the data

In [5]:
query_name = '''
SELECT
    olf.country_code                                 AS country_code,
    olf.city_code                                    AS city_code,
    olf.order_id                                     AS order_id,
    olf.courier_id                                   AS courier_id,
    olf.order_created_local_datetime                 AS creation_timestamp,
    olf.order_activated_local_datetime               AS activation_timestamp,
    olf.courier_transport                            AS transport,
    olf.order_picked_up_local_datetime               AS pickup_timestamp,
    olf.order_delivered_local_datetime               AS delivery_timestamp,
    olf.order_pickup_latitude                        AS pickup_latitude,
    olf.order_pickup_longitude                       AS pickup_longitude,
    olf.order_delivery_latitude                      AS delivery_latitude,
    olf.order_delivery_longitude                     AS delivery_longitude,
    olf.order_arrival_to_delivery_local_datetime     AS delivery_entering_timestamp,
    olf.order_time_zone                              AS time_zone,
    olf.p_creation_date
FROM delta.courier_routing_courier_ml_features_odp.order_level_features AS olf
WHERE order_final_status = 'DeliveredStatus'
    AND order_number_of_assignments = 1
    AND order_bundle_index IS NULL
    AND p_creation_date >= DATE '[start_date]' AND p_creation_date < DATE '[end_date]'
    AND country_code IN ('[country_code]')
    AND city_code IN ([cities])
'''

query = qr.Query(base_query_path, query_name, datalake_connection, parameters_dict=parameters, query_from_file = False)

df = query.run()
df = df.fillna(value=np.nan)

data = df.copy()
data.head()

Open the following URL in browser for the external authentication:
https://starburst.g8s-data-platform-prod.glovoint.com/oauth2/token/initiate/ddb626736a4a4e375dfc8ad34af7842030a2b1b275912186850c26d29aa7b046


,country_code,city_code,order_id,courier_id,creation_timestamp,activation_timestamp,transport,pickup_timestamp,delivery_timestamp,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,delivery_entering_timestamp,time_zone,p_creation_date
0,ES,BCN,100907487116,8590944,2024-10-03 20:55:02+00:00,2024-10-03 20:55:04+00:00,BICYCLE,2024-10-03 21:21:44.215000+00:00,2024-10-03 21:32:33+00:00,41.370594,2.113433,41.375412,2.104837,2024-10-03 21:29:37+00:00,Europe/Madrid,2024-10-03
1,ES,MAD,100907489425,162535288,2024-10-03 20:55:43+00:00,2024-10-03 20:55:44+00:00,BICYCLE,2024-10-03 21:07:40.941000+00:00,2024-10-03 21:19:10+00:00,40.382927,-3.626392,40.383490,-3.640969,2024-10-03 21:16:09+00:00,Europe/Madrid,2024-10-03
2,ES,BCN,100907493124,146788593,2024-10-03 20:56:48+00:00,2024-10-03 20:56:49+00:00,BICYCLE,2024-10-03 21:07:40.560000+00:00,2024-10-03 21:14:30+00:00,41.389874,2.159761,41.390520,2.148718,2024-10-03 21:12:00+00:00,Europe/Madrid,2024-10-03
3,ES,MAD,100907498582,45695767,2024-10-03 20:58:25+00:00,2024-10-03 20:58:26+00:00,MOTORBIKE,2024-10-03 21:08:59.224000+00:00,2024-10-03 21:14:21+00:00,40.409720,-3.670494,40.412880,-3.671840,2024-10-03 21:11:21+00:00,Europe/Madrid,2024-10-03
4,ES,MAD,100907503468,166385377,2024-10-03 20:59:51+00:00,2024-10-03 20:59:53+00:00,MOTORBIKE,2024-10-03 21:09:49.060000+00:00,2024-10-03 21:20:30+00:00,40.433780,-3.683620,40.430176,-3.673502,2024-10-03 21:16:45+00:00,Europe/Madrid,2024-10-03


## Clean the dataset

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 987012 entries, 0 to 87011
Data columns (total 16 columns):
 #   Column                       Non-Null Count   Dtype              
---  ------                       --------------   -----              
 0   country_code                 987012 non-null  object             
 1   city_code                    987012 non-null  object             
 2   order_id                     987012 non-null  int64              
 3   courier_id                   987012 non-null  int64              
 4   creation_timestamp           987012 non-null  datetime64[ns, UTC]
 5   activation_timestamp         987012 non-null  datetime64[ns, UTC]
 6   transport                    987012 non-null  object             
 7   pickup_timestamp             986757 non-null  datetime64[ns, UTC]
 8   delivery_timestamp           987012 non-null  datetime64[ns, UTC]
 9   pickup_latitude              987012 non-null  float64            
 10  pickup_longitude             987012 no

In [7]:
data.describe()

,order_id,courier_id,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude
count,9.870120e+05,9.870120e+05,987012.000000,987012.000000,987012.000000,987012.000000
mean,1.009222e+11,1.218706e+08,40.673782,-0.851254,40.674140,-0.851210
std,1.301628e+07,5.590163e+07,0.993792,3.003208,0.993963,3.002620
min,1.008998e+11,1.127770e+05,37.282646,-6.085188,37.260975,-6.100339
25%,1.009108e+11,7.241826e+07,40.426132,-3.690008,40.426205,-3.689353
50%,1.009222e+11,1.482987e+08,40.530730,-0.483401,40.528835,-0.483956
75%,1.009337e+11,1.670841e+08,41.393280,2.159965,41.394840,2.159824
max,1.009444e+11,1.796540e+08,41.483086,2.268973,41.562153,2.277824


In [8]:
# Check for missing values
data.isnull().sum()

country_code                       0
city_code                          0
order_id                           0
courier_id                         0
creation_timestamp                 0
activation_timestamp               0
transport                          0
pickup_timestamp                 255
delivery_timestamp                 0
pickup_latitude                    0
pickup_longitude                   0
delivery_latitude                  0
delivery_longitude                 0
delivery_entering_timestamp    14541
time_zone                          0
p_creation_date                    0
dtype: int64

In [9]:
# Remove rows with null values: we have 1.5 M rows, so we can afford to remove them
data = data.dropna()

In [10]:
# Check for missing values
data.isnull().sum()

country_code                   0
city_code                      0
order_id                       0
courier_id                     0
creation_timestamp             0
activation_timestamp           0
transport                      0
pickup_timestamp               0
delivery_timestamp             0
pickup_latitude                0
pickup_longitude               0
delivery_latitude              0
delivery_longitude             0
delivery_entering_timestamp    0
time_zone                      0
p_creation_date                0
dtype: int64

In [11]:
# Check for duplicates
data.duplicated().sum()

np.int64(0)

## Compute new features

In [12]:
# Convert the creation time to datetime
data['creation_timestamp'] = pd.to_datetime(data['creation_timestamp'])
data['activation_timestamp'] = pd.to_datetime(data['activation_timestamp'])
data['pickup_timestamp'] = pd.to_datetime(data['pickup_timestamp'])
data['delivery_timestamp'] = pd.to_datetime(data['delivery_timestamp'])
data['delivery_entering_timestamp'] = pd.to_datetime(data['delivery_entering_timestamp'])

# Compute the delivery date and the delivery time
data['creation_date'] = data['creation_timestamp'].dt.date
data['creation_time'] = data['creation_timestamp'].dt.time
data['creation_hour'] = data['creation_timestamp'].dt.hour

In [24]:
# Compute the distance between the pickup and delivery points
data['pd_distance_haversine_m'] = data.apply(lambda x: haversine((x['pickup_latitude'], x['pickup_longitude']), (x['delivery_latitude'], x['delivery_longitude']), unit=Unit.METERS), axis=1)
data['pd_distance_haversine_m_sk'] = data.apply(lambda x: haversine_distances(np.array([[x['pickup_latitude'], x['pickup_longitude']]]), np.array([[x['delivery_latitude'], x['delivery_longitude']]])), axis=1)
data['pd_distance_manhattan_m'] = data.apply(lambda x: manhattan_distances(np.array([[x['pickup_latitude'], x['pickup_longitude']]]), np.array([[x['delivery_latitude'], x['delivery_longitude']]])), axis=1)
data.head()

,country_code,city_code,order_id,courier_id,creation_timestamp,activation_timestamp,transport,pickup_timestamp,delivery_timestamp,pickup_latitude,...,delivery_longitude,delivery_entering_timestamp,time_zone,p_creation_date,creation_date,creation_time,creation_hour,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m
0,ES,BCN,100907487116,8590944,2024-10-03 20:55:02+00:00,2024-10-03 20:55:04+00:00,BICYCLE,2024-10-03 21:21:44.215000+00:00,2024-10-03 21:32:33+00:00,41.370594,...,2.104837,2024-10-03 21:29:37+00:00,Europe/Madrid,2024-10-03,2024-10-03,20:55:02,20,895.267385,[[0.008835822216147426]],[[0.013414000000000481]]
1,ES,MAD,100907489425,162535288,2024-10-03 20:55:43+00:00,2024-10-03 20:55:44+00:00,BICYCLE,2024-10-03 21:07:40.941000+00:00,2024-10-03 21:19:10+00:00,40.382927,...,-3.640969,2024-10-03 21:16:09+00:00,Europe/Madrid,2024-10-03,2024-10-03,20:55:43,20,1236.264251,[[0.013090010777783042]],[[0.01513999999999971]]
2,ES,BCN,100907493124,146788593,2024-10-03 20:56:48+00:00,2024-10-03 20:56:49+00:00,BICYCLE,2024-10-03 21:07:40.560000+00:00,2024-10-03 21:14:30+00:00,41.389874,...,2.148718,2024-10-03 21:12:00+00:00,Europe/Madrid,2024-10-03,2024-10-03,20:56:48,20,924.017061,[[0.009439473750393588]],[[0.011689000000003169]]
3,ES,MAD,100907498582,45695767,2024-10-03 20:58:25+00:00,2024-10-03 20:58:26+00:00,MOTORBIKE,2024-10-03 21:08:59.224000+00:00,2024-10-03 21:14:21+00:00,40.409720,...,-3.671840,2024-10-03 21:11:21+00:00,Europe/Madrid,2024-10-03,2024-10-03,20:58:25,20,369.394256,[[0.003388700168894049]],[[0.004506000000001009]]
4,ES,MAD,100907503468,166385377,2024-10-03 20:59:51+00:00,2024-10-03 20:59:53+00:00,MOTORBIKE,2024-10-03 21:09:49.060000+00:00,2024-10-03 21:20:30+00:00,40.433780,...,-3.673502,2024-10-03 21:16:45+00:00,Europe/Madrid,2024-10-03,2024-10-03,20:59:51,20,945.506113,[[0.009959495756425248]],[[0.013721999999995571]]


There is a difference in how the `haversine` library, the `sklearn`'s `haversine_distances`, and the `sklearn`'s `manhattan_distances` function compute and return the distances. Let's break down the differences and how to resolve them:
1. **Haversine Library:**
   - The `haversine` library directly computes the distance between two points and returns a single scalar value.
2. **Sklearn's `haversine_distances`:**
   - The `haversine_distances` function from `sklearn` returns a distance matrix. When you input two points, it returns a 1x1 matrix (a nested list) containing the distance. This is why you would see the result in squared parentheses like `[[]]`. We extract the single value using `[0][0]`.
   - Additionally, the `haversine_distances` function returns the distance in radians, not in meters. To convert this to meters, you need to multiply by the Earth's radius (approximately 6371000 meters).
3. **Sklearn's `manhattan_distances`:**
   - The `manhattan_distances` function computes the Manhattan distance between two points and returns a distance matrix. We extract the single value from the 1x1 matrix using `[0][0]`.
   - Additionally, the `manhattan_distances` function from sklearn computes the distance based on the Cartesian coordinates provided. Since latitude and longitude are angular measurements, the result will not be in meters but in degrees. To convert the Manhattan distance from degrees to meters, you need to account for the Earth's curvature.

In [27]:
# Earth's radius in meters
earth_radius_m = 6371.0088 * 1000

# Conversion factors
meters_per_degree_lat = 111320  # Approximate meters per degree of latitude

def manhattan_distance_in_meters(row):
    # Convert latitude and longitude differences to meters
    lat_diff_m = abs(row['pickup_latitude'] - row['delivery_latitude']) * meters_per_degree_lat
    # Convert longitude difference to meters, considering the latitude
    lon_diff_m = abs(row['pickup_longitude'] - row['delivery_longitude']) * meters_per_degree_lat * np.cos(np.radians((row['pickup_latitude'] + row['delivery_latitude']) / 2))
    # Sum the absolute differences to get the Manhattan distance in meters
    return lat_diff_m + lon_diff_m

In [28]:
data['pd_distance_haversine_m'] = data.apply(
    lambda x: haversine(
        (x['pickup_latitude'], x['pickup_longitude']),
        (x['delivery_latitude'], x['delivery_longitude']),
        unit=Unit.METERS
    ), axis=1
)
data['pd_distance_haversine_m_sk'] = data.apply(
    lambda x: haversine_distances(
        np.array([[x['pickup_latitude'], x['pickup_longitude']]]),
        np.array([[x['delivery_latitude'], x['delivery_longitude']]])
    )[0][0] * earth_radius_m, axis=1
)
data['pd_distance_manhattan_m'] = data.apply(manhattan_distance_in_meters, axis=1)
data.head()

,country_code,city_code,order_id,courier_id,creation_timestamp,activation_timestamp,transport,pickup_timestamp,delivery_timestamp,pickup_latitude,...,delivery_entering_timestamp,time_zone,p_creation_date,creation_date,creation_time,creation_hour,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,pd_distance_manhattan
0,ES,BCN,100907487116,8590944,2024-10-03 20:55:02+00:00,2024-10-03 20:55:04+00:00,BICYCLE,2024-10-03 21:21:44.215000+00:00,2024-10-03 21:32:33+00:00,41.370594,...,2024-10-03 21:29:37+00:00,Europe/Madrid,2024-10-03,2024-10-03,20:55:02,20,895.267385,56293.101094,1254.424177,0.013414
1,ES,MAD,100907489425,162535288,2024-10-03 20:55:43+00:00,2024-10-03 20:55:44+00:00,BICYCLE,2024-10-03 21:07:40.941000+00:00,2024-10-03 21:19:10+00:00,40.382927,...,2024-10-03 21:16:09+00:00,Europe/Madrid,2024-10-03,2024-10-03,20:55:43,20,1236.264251,83396.573857,1298.738405,0.015140
2,ES,BCN,100907493124,146788593,2024-10-03 20:56:48+00:00,2024-10-03 20:56:49+00:00,BICYCLE,2024-10-03 21:07:40.560000+00:00,2024-10-03 21:14:30+00:00,41.389874,...,2024-10-03 21:12:00+00:00,Europe/Madrid,2024-10-03,2024-10-03,20:56:48,20,924.017061,60138.970331,994.168407,0.011689
3,ES,MAD,100907498582,45695767,2024-10-03 20:58:25+00:00,2024-10-03 20:58:26+00:00,MOTORBIKE,2024-10-03 21:08:59.224000+00:00,2024-10-03 21:14:21+00:00,40.409720,...,2024-10-03 21:11:21+00:00,Europe/Madrid,2024-10-03,2024-10-03,20:58:25,20,369.394256,21589.438597,465.858447,0.004506
4,ES,MAD,100907503468,166385377,2024-10-03 20:59:51+00:00,2024-10-03 20:59:53+00:00,MOTORBIKE,2024-10-03 21:09:49.060000+00:00,2024-10-03 21:20:30+00:00,40.433780,...,2024-10-03 21:16:45+00:00,Europe/Madrid,2024-10-03,2024-10-03,20:59:51,20,945.506113,63452.035108,1258.537546,0.013722


## Save the dataset

It's better to use the parquet format, as it is more efficient and faster to read and write. Besides, it is a columnar format, which is more suitable for analytical queries. We can also partition the data by city and creation date, which will help to speed up the queries.

In [ ]:
data.to_parquet("data/dataframe.parquet", partition_cols=['creation_date', 'city_code'])

## Exploratory Data Analysis (EDA)

In [ ]:
data = pd.read_parquet("data/dataframe.parquet")

In [ ]:
# Histogram of the # of data per day / hour
plt.figure(figsize=(15, 8))
plt.hist(data['creation_timestamp'], bins = 1000)
plt.title('Histogram of the # of data per day / hour')
plt.xlabel('Day / Hour')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(data['creation_date'], bins = 14)
plt.title('Histogram of the # of data per day')
plt.xlabel('Day')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(data['creation_hour'])
plt.title('Histogram of the # of data per hour')
plt.xlabel('Hour')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Check the distribution of the transport types
data['transport'].value_counts()

In [ ]:
# Check the distribution of the distances
plt.figure(figsize=(15, 8))
plt.hist(data['pd_distance_haversine_m'], bins = 1000)
plt.xlim(0, 10000)
plt.title('Histogram of the distances')
plt.xlabel('Distance (m)')
plt.ylabel('Frequency')
plt.show()

## Hyperparameters

In [ ]:
test_set_perc = 0.1
days_for_test = 7
k_cv = 5

## Database split

In [ ]:
X = data
y = data['delivery_entering_timestamp'] - data['pickup_timestamp']
y

As we are dealing with a time-series dataset (orders are placed at different times), we will split the data based on the creation timestamp, leaving out the last 10% of the data for testing. This will help to understand the performance of the model on unseen data, as in reality we will have to test the model on data created on day+1 with respect to our training data.

In [ ]:
X.sort_values('creation_timestamp', inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_perc, random_state=0)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# In case we want to test different hyperparameters, we will use cross-validation
#scores = cross_val_score(<estimator>, X, y, cv=k_cv)

### Database split using directly the creation date

As we have partitioned the data by city and creation date, we can use this information to split the data. This will help to avoid data leakage, as we will not have data from the future in the training set.
This is much better than just sorting the data by the creation timestamp and taking 10% of the dataset as test set, as we did before.

In [ ]:
begin_test_date = X['creation_date'].unique()[-int(test_set_perc * len(X['creation_date'].unique()))]
begin_test_date

In [ ]:
# We take the last week of the dataset to test the model
begin_test_date = end_date - pd.Timedelta(days=days_for_test)

In [ ]:
X_train = pd.read_parquet("data/dataframe.parquet", filters=[('creation_date', '<=', begin_test_date)])

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test

Compute the column for velocity 

In [ ]:
X_train['velocity'] = X_train['pd_distance_haversine_m'] / y_train.dt.total_seconds()